In [1]:
# Fisrt:convert yolov3-tiny cfg to caffe prototxt
# Secnd:convert yolov3-tiny weights to caffe caffemodel 
# Final:use ncs model optimizer to generate corespending xml and bin
# Note: For NCS1 to Generate fixed 6classes graph , Please use another file named 'do_convert_ncs1_ncs2_new.py'.
# We use NCS1 just 6 classes before , so the code is not fit for other generally classes condition.

In [3]:
import os, sys, datetime
print(sys.path)
sys.path.insert(0,'/data/ssd-caffe/py2_caffe/python/')
import caffe  
import numpy as np
from collections import OrderedDict
from ConfigParser import ConfigParser

class UniqDict(OrderedDict):
    _unique = 0
    def __setitem__(self, key, val):
        if isinstance(val, OrderedDict):
            self._unique += 1
            key += "_"+str(self._unique)
        OrderedDict.__setitem__(self, key, val)
def parse_cfg(cfgfile):
    parser = ConfigParser(dict_type=UniqDict)
    parser.read(cfgfile)
    blocks = parser.sections()
    return blocks, parser
def is_number(s):
    try:
        float(s)
        return True
    except ValueError:
        return False
def cfg2prototxt(cfgfile):
    blocks, parser = parse_cfg(cfgfile)
    print(blocks)
    layers = []  
    props = OrderedDict()  
    bottom = 'data'  
    layer_id = 1  
    topnames = dict()  
    for block in blocks:
        items = dict(parser.items(block))
        items['type'] = block.split('_')[0]
        print(items)
        if items['type'] == 'net':  
            props['name'] = 'Darkent2Caffe'  
            props['input'] = 'data'  
            props['input_dim'] = ['1']  
            props['input_dim'].append(items['channels'])  
            props['input_dim'].append(items['height'])  
            props['input_dim'].append(items['width'])  
            continue  
        elif (items['type'] == 'convolutional' or items['type'] == 'deconvolutional'):  
            conv_layer = OrderedDict()  
            conv_layer['bottom'] = bottom  
            if items.has_key('name'):  
                conv_layer['top'] = items['name']  
                conv_layer['name'] = items['name']  
            else:
                if(items['type'] == 'convolutional'):
                    conv_layer['top'] = 'layer%d-conv' % layer_id  
                    conv_layer['name'] = 'layer%d-conv' % layer_id
                elif(items['type'] == 'deconvolutional'):
                    conv_layer['top'] = 'layer%d-upsample' % layer_id  
                    conv_layer['name'] = 'layer%d-upsample' % layer_id
                    
            if(items['type'] == 'deconvolutional'):
                conv_layer['type'] = 'Deconvolution'
            elif(items['type'] == 'convolutional'):
                conv_layer['type'] = 'Convolution' 
            convolution_param = OrderedDict()  
            convolution_param['num_output'] = items['filters']  
            convolution_param['kernel_size'] = items['size']  
            if items['pad'] == '1':  
                convolution_param['pad'] = str(int(convolution_param['kernel_size'])/2)  
            convolution_param['stride'] = items['stride']
            if items.has_key('batch_normalize'):
                if items['batch_normalize'] == '1':  
                    convolution_param['bias_term'] = 'false' 
            else:  
                convolution_param['bias_term'] = 'true'  
            conv_layer['convolution_param'] = convolution_param  
            layers.append(conv_layer)  
            bottom = conv_layer['top']  
            if items.has_key('batch_normalize'):
                if items['batch_normalize'] == '1': 
                    bn_layer = OrderedDict()  
                    bn_layer['bottom'] = bottom  
                    bn_layer['top'] = bottom  
                    if items.has_key('name'):  
                        bn_layer['name'] = '%s-bn' % items['name']  
                    else:  
                        bn_layer['name'] = 'layer%d-bn' % layer_id  
                    bn_layer['type'] = 'BatchNorm'  
                    batch_norm_param = OrderedDict()  
                    batch_norm_param['use_global_stats'] = 'true'  
                    bn_layer['batch_norm_param'] = batch_norm_param  
                    layers.append(bn_layer)  

                    scale_layer = OrderedDict()  
                    scale_layer['bottom'] = bottom  
                    scale_layer['top'] = bottom  
                    if items.has_key('name'):  
                        scale_layer['name'] = '%s-scale' % items['name']  
                    else:  
                        scale_layer['name'] = 'layer%d-scale' % layer_id  
                    scale_layer['type'] = 'Scale'  
                    scale_param = OrderedDict()  
                    scale_param['bias_term'] = 'true'  
                    scale_layer['scale_param'] = scale_param  
                    layers.append(scale_layer)  

            if items['activation'] != 'linear':  
                relu_layer = OrderedDict()  
                relu_layer['bottom'] = bottom  
                relu_layer['top'] = bottom  
                if items.has_key('name'):  
                    relu_layer['name'] = '%s-act' % items['name']  
                else:  
                    relu_layer['name'] = 'layer%d-act' % layer_id  
                relu_layer['type'] = 'ReLU'  
                if items['activation'] == 'leaky':  
                    relu_param = OrderedDict()  
                    relu_param['negative_slope'] = '0.1'  
                    relu_layer['relu_param'] = relu_param  
                layers.append(relu_layer)  
            topnames[layer_id] = bottom  
            layer_id = layer_id+1  
        elif items['type'] == 'maxpool':  
            max_layer = OrderedDict()  
            max_layer['bottom'] = bottom  
            if items.has_key('name'):  
                max_layer['top'] = items['name']  
                max_layer['name'] = items['name']  
            else:  
                max_layer['top'] = 'layer%d-maxpool' % layer_id  
                max_layer['name'] = 'layer%d-maxpool' % layer_id  
            max_layer['type'] = 'Pooling'  
            pooling_param = OrderedDict()  
            pooling_param['kernel_size'] = items['size']  
            pooling_param['stride'] = items['stride']  
            pooling_param['pool'] = 'MAX'  
            if items.has_key('pad') and int(items['pad']) == 1:  
                pooling_param['pad'] = str((int(items['size'])-1)/2)  
            max_layer['pooling_param'] = pooling_param  
            layers.append(max_layer)  
            bottom = max_layer['top']  
            topnames[layer_id] = bottom  
            layer_id = layer_id+1  
        elif items['type'] == 'avgpool':  
            avg_layer = OrderedDict()  
            avg_layer['bottom'] = bottom  
            if block.has_key('name'):  
                avg_layer['top'] = items['name']  
                avg_layer['name'] = items['name']  
            else:  
                avg_layer['top'] = 'layer%d-avgpool' % layer_id  
                avg_layer['name'] = 'layer%d-avgpool' % layer_id  
            avg_layer['type'] = 'Pooling'  
            pooling_param = OrderedDict()  
            pooling_param['kernel_size'] = 7  
            pooling_param['stride'] = 1  
            pooling_param['pool'] = 'AVE'  
            avg_layer['pooling_param'] = pooling_param  
            layers.append(avg_layer)  
            bottom = avg_layer['top']  
            topnames[layer_id] = bottom  
            layer_id = layer_id+1  
        elif items['type'] == 'yolo': 
            layer_id = layer_id + 1
            continue
            region_layer = OrderedDict()  
            region_layer['bottom'] = bottom  
            if items.has_key('name'):  
                region_layer['top'] = items['name']  
                region_layer['name'] = items['name']  
            else:  
                region_layer['top'] = 'layer%d-yolo' % layer_id  
                region_layer['name'] = 'layer%d-yolo' % layer_id  
            region_layer['type'] = 'Yolo'  
            region_param = OrderedDict()  
            region_param['anchors'] = items['anchors'].strip()  
            region_param['classes'] = items['classes']  
            region_param['num'] = items['num']  
            region_layer['yolo_param'] = region_param  
            layers.append(region_layer)  
            bottom = region_layer['top']  
            topnames[layer_id] = bottom  
            layer_id = layer_id + 1

        elif items['type'] == 'route':
            route_layer = OrderedDict()  
            layer_name = str(items['layers']).split(',')  
            print(layer_name[0])  
            bottom_layer_size = len(str(items['layers']).split(','))  
            #print(bottom_layer_size)  
            if(1 == bottom_layer_size):  
                prev_layer_id = layer_id + int(items['layers'])  
                bottom = topnames[prev_layer_id]  
                #topnames[layer_id] = bottom  
            route_layer['bottom'] = bottom  
            if(2 == bottom_layer_size):  
                prev_layer_id1 = layer_id + int(layer_name[0])  
                #print(prev_layer_id1)  
                prev_layer_id2 = int(layer_name[1]) + 1  
                print(topnames)  
                bottom1 = topnames[prev_layer_id1]  
                bottom2 = topnames[prev_layer_id2]  
                route_layer['bottom'] = [bottom1, bottom2]  
            if items.has_key('name'):  
                route_layer['top'] = items['name']  
                route_layer['name'] = items['name']  
            else:  
                route_layer['top'] = 'layer%d-route' % layer_id  
                route_layer['name'] = 'layer%d-route' % layer_id  
            route_layer['type'] = 'Concat'
            print(route_layer)  
            layers.append(route_layer)  
            bottom = route_layer['top']  
            print(layer_id)  
            topnames[layer_id] = bottom  
            layer_id = layer_id + 1  

        elif items['type'] == 'upsample':
            upsample_layer = OrderedDict()  
            print(items['stride'])  
            upsample_layer['bottom'] = bottom  
            if items.has_key('name'):  
                upsample_layer['top'] = items['name']  
                upsample_layer['name'] = items['name']  
            else:  
                upsample_layer['top'] = 'layer%d-upsample' % layer_id  
                upsample_layer['name'] = 'layer%d-upsample' % layer_id  
            upsample_layer['type'] = 'Upsample'  
            upsample_param = OrderedDict()  
            upsample_param['scale'] = items['stride']  
            upsample_layer['upsample_param'] = upsample_param  
            print(upsample_layer)  
            layers.append(upsample_layer)  
            bottom = upsample_layer['top']  
            print('upsample:',layer_id)  
            topnames[layer_id] = bottom  
            layer_id = layer_id + 1  

        elif items['type'] == 'shortcut':  
            prev_layer_id1 = layer_id + int(items['from'])  
            prev_layer_id2 = layer_id - 1  
            bottom1 = topnames[prev_layer_id1]  
            bottom2= topnames[prev_layer_id2]  
            shortcut_layer = OrderedDict()  
            shortcut_layer['bottom'] = [bottom1, bottom2]  
            if items.has_key('name'):  
                shortcut_layer['top'] = items['name']  
                shortcut_layer['name'] = items['name']  
            else:  
                shortcut_layer['top'] = 'layer%d-shortcut' % layer_id  
                shortcut_layer['name'] = 'layer%d-shortcut' % layer_id  
            shortcut_layer['type'] = 'Eltwise'
            eltwise_param = OrderedDict()  
            eltwise_param['operation'] = 'SUM'  
            shortcut_layer['eltwise_param'] = eltwise_param  
            layers.append(shortcut_layer)  
            bottom = shortcut_layer['top']  

            if items['activation'] != 'linear':  
                relu_layer = OrderedDict()  
                relu_layer['bottom'] = bottom  
                relu_layer['top'] = bottom  
                if block.has_key('name'):  
                    relu_layer['name'] = '%s-act' % items['name']  
                else:  
                    relu_layer['name'] = 'layer%d-act' % layer_id  
                relu_layer['type'] = 'ReLU'  
                if items['activation'] == 'leaky':  
                    relu_param = OrderedDict()  
                    relu_param['negative_slope'] = '0.1'  
                    relu_layer['relu_param'] = relu_param  
                layers.append(relu_layer)  
            topnames[layer_id] = bottom  
            layer_id = layer_id + 1             

        elif items['type'] == 'connected':  
            fc_layer = OrderedDict()  
            fc_layer['bottom'] = bottom  
            if items.has_key('name'):  
                fc_layer['top'] = items['name']  
                fc_layer['name'] = items['name']  
            else:  
                fc_layer['top'] = 'layer%d-fc' % layer_id  
                fc_layer['name'] = 'layer%d-fc' % layer_id  
            fc_layer['type'] = 'InnerProduct'  
            fc_param = OrderedDict()  
            fc_param['num_output'] = int(items['output'])  
            fc_layer['inner_product_param'] = fc_param  
            layers.append(fc_layer)  
            bottom = fc_layer['top']  

            if items['activation'] != 'linear':  
                relu_layer = OrderedDict()  
                relu_layer['bottom'] = bottom  
                relu_layer['top'] = bottom  
                if items.has_key('name'):  
                    relu_layer['name'] = '%s-act' % items['name']  
                else:  
                    relu_layer['name'] = 'layer%d-act' % layer_id  
                relu_layer['type'] = 'ReLU'  
                if items['activation'] == 'leaky':  
                    relu_param = OrderedDict()  
                    relu_param['negative_slope'] = '0.1'  
                    relu_layer['relu_param'] = relu_param  
                layers.append(relu_layer)  
            topnames[layer_id] = bottom  
            layer_id = layer_id+1  
        else:  
            print('unknow layer type %s ' % items['type'])  
            topnames[layer_id] = bottom  
            layer_id = layer_id + 1  

    net_info = OrderedDict()  
    net_info['props'] = props  
    net_info['layers'] = layers  
    return net_info

def save_prototxt(net_info, protofile, yolo=False):
    fp = open(protofile, 'w')
    # whether add double quote
    def format_value(value):
        #str = u'%s' % value
        #if str.isnumeric():
        if is_number(value):
            return value
        elif value == 'true' or value == 'false' or value == 'MAX' or value == 'SUM' or value == 'AVE':
            return value
        else:
            return '\"%s\"' % value

    def print_block(block_info, prefix, indent):
        blanks = ''.join([' ']*indent)
        print >>fp, '%s%s {' % (blanks, prefix)
        for key,value in block_info.items():
            if type(value) == OrderedDict:
                print_block(value, key, indent+4)
            elif type(value) == list:
                for v in value:
                    print >> fp, '%s    %s: %s' % (blanks, key, format_value(v))
            else:
                print >> fp, '%s    %s: %s' % (blanks, key, format_value(value))
        print >> fp, '%s}' % blanks
        
    props = net_info['props']
    layers = net_info['layers']
    print >> fp, 'name: \"%s\"' % props['name']
    print >> fp, 'input: \"%s\"' % props['input']
    print >> fp, 'input_shape{'
    print >> fp, '  dim: %s' % props['input_dim'][0]
    print >> fp, '  dim: %s' % props['input_dim'][1]
    print >> fp, '  dim: %s' % props['input_dim'][2]
    print >> fp, '  dim: %s' % props['input_dim'][3]
    print >> fp, '}'
    #print >> fp, ''
    for layer in layers:
        if layer['type'] != 'yolo' or yolo == True:
            print_block(layer, 'layer', 0)
    fp.close()
cfgfile = '/data/github_repos/darknet/ncs/yolov3-tiny-ncs-without-last-maxpool.cfg'
cfgfile = '/data/github_repos/darknet/ncs/yolov3-tiny-ncs-without-last-maxpool-refined-anchors-7cls-test.cfg'
cfgfile = '/data/github_repos/bigfile/yolov3-tiny-ncs-without-last-maxpool-refined-anchors-3cls-extend-\
#remove-maxpooling_new_test_864.cfg'
cfgfile = '/media/yyp/DATA/EVALUATE/yolov3-tiny-ncs-without-last-maxpool-refined-anchors-7cls-test.cfg'
#cfgfile = '/media/yyp/DATA/EVALUATE/yolov3-tiny-ncs-without-last-maxpool-refined-anchors-7cls-test.cfg'
#cfgfile = '/media/yyp/DATA/EVALUATE/yolov3-tiny-ncs-without-last-maxpool-refined-anchors-6cls-new-refine-\
#shortcut-test.cfg'
#cfgfile = '/media/yyp/DATA/EVALUATE/yolov3-tiny-ncs-without-last-maxpool-refined-anchors-4cls-test.cfg'
saved_prototxt = str(datetime.datetime.now()).split(' ')[-1] + cfgfile.split('/')[-1] + '.prototxt'
net_info = cfg2prototxt(cfgfile)    
save_prototxt(net_info, saved_prototxt)

['/data/ssd-caffe/py2_upsample_caffe/python', '/data/ssd-caffe/py2_caffe/python/', '', '/opt/intel/openvino_2019.2.242/python/python3.5', '/opt/intel/openvino_2019.2.242/python/python3', '/opt/intel/openvino_2019.2.242/deployment_tools/model_optimizer', '/data/github_repos/yolov3-tiny-fit-ncs/ncs/convert_tools', '/data/ssd-caffe/caffe/python', '/usr/lib/python2.7', '/usr/lib/python2.7/plat-x86_64-linux-gnu', '/usr/lib/python2.7/lib-tk', '/usr/lib/python2.7/lib-old', '/usr/lib/python2.7/lib-dynload', '/home/yyp/.local/lib/python2.7/site-packages', '/usr/local/lib/python2.7/dist-packages', '/usr/lib/python2.7/dist-packages', '/usr/lib/python2.7/dist-packages/gtk-2.0', '/home/yyp/.local/lib/python2.7/site-packages/IPython/extensions', '/home/yyp/.ipython']
['net_1', 'convolutional_2', 'maxpool_3', 'convolutional_4', 'maxpool_5', 'convolutional_6', 'maxpool_7', 'convolutional_8', 'maxpool_9', 'convolutional_10', 'maxpool_11', 'convolutional_12', 'convolutional_13', 'convolutional_14', 'con

In [5]:
'''
if(len(sys.argv) < 2):
    print('please input yolov3-tiny-ncs-weights file')
    exit(1)
weights = sys.argv[1]
'''
#weights = '/media/yyp/DATA/EVALUATE/yolov3-tiny-ncs-without-last-maxpool-refined-anchors_150000.weights'
weights = '/media/yyp/DATA/EVALUATE/yolov3-tiny-ncs-without-last-maxpool-refined-anchors-7cls_130000.weights'
#weights = '/data/github_repos/bigfile/yolov3-tiny-ncs-without-last-maxpool-refined-anchors-3cls-extend-\
#remove-maxpooling_new.backup'
#weights = '/media/yyp/DATA/EVALUATE/yolov3-tiny-ncs-without-last-maxpool-refined-anchors-4cls-new-\
#refine-shortcut.backup'
#weights = '/media/yyp/DATA/EVALUATE/yolov3-tiny-ncs-without-last-maxpool-refined-anchors-7cls-haikou-person.backup'
#weights = '/media/yyp/DATA/EVALUATE/yolov3-tiny-ncs-without-last-maxpool-refined-anchors-6cls-new-refine-\
#shortcut_18000.weights'
#weights = '/media/yyp/DATA/EVALUATE/yolov3-tiny-ncs-without-last-maxpool-refined-anchors-4cls_100000.weights'
#weights = '/nfsroot/yyp-darknet/darknet/zeshida_weights_20200523/\
#yolov3-tiny-ncs-without-last-maxpool-refined-anchors-4cls.backup'
weights = '/nfsroot/yyp-darknet/darknet/yolov3_tiny_20200525_weights_lsw/\
yolov3-tiny-ncs-without-last-maxpool-refined-anchors-7cls_100000.weights'
#sys.path.insert(0, '/data/ssd-caffe/py2_caffe/python') 
#sys.path.insert(0, '/data/ssd-caffe/new-yolov3-caffe/python')
sys.path.insert(0, '/data/ssd-caffe/py2_upsample_caffe/python')
del caffe
import caffe  
import numpy as np  
from collections import OrderedDict
from ConfigParser import ConfigParser

#from ConfigParser import ConfigParser
class uniqdict(OrderedDict):
    _unique = 0
    def __setitem__(self, key, val):
        if isinstance(val, OrderedDict):
            self._unique += 1
            key += "_"+str(self._unique)
        OrderedDict.__setitem__(self, key, val)
def load_conv2caffe(buf, start, conv_param):  
    weight = conv_param[0].data  
    bias = conv_param[1].data  
    conv_param[1].data[...] = np.reshape(buf[start:start+bias.size], bias.shape);   start = start + bias.size  
    conv_param[0].data[...] = np.reshape(buf[start:start+weight.size], weight.shape); start = start + weight.size  
    return start
def load_fc2caffe(buf, start, fc_param):  
    weight = fc_param[0].data  
    bias = fc_param[1].data  
    fc_param[1].data[...] = np.reshape(buf[start:start+bias.size], bias.shape);   start = start + bias.size  
    fc_param[0].data[...] = np.reshape(buf[start:start+weight.size], weight.shape); start = start + weight.size  
    return start
def load_conv_bn2caffe(buf, start, conv_param, bn_param, scale_param): 
    conv_weight = conv_param[0].data  
    running_mean = bn_param[0].data  
    running_var = bn_param[1].data  
    scale_weight = scale_param[0].data  
    scale_bias = scale_param[1].data      
    scale_param[1].data[...] = np.reshape(buf[start:start+scale_bias.size], scale_bias.shape); start = start + scale_bias.size  
    #print scale_bias.size  
    #print scale_bias  
  
    scale_param[0].data[...] = np.reshape(buf[start:start+scale_weight.size], scale_weight.shape); start = start + scale_weight.size  
    #print scale_weight.size
    #print scale_weight
  
    bn_param[0].data[...] = np.reshape(buf[start:start+running_mean.size], running_mean.shape); start = start + running_mean.size  
    #print running_mean.size
    #print running_mean
  
    bn_param[1].data[...] = np.reshape(buf[start:start+running_var.size], running_var.shape); start = start + running_var.size  
    #print running_var.size
    #print running_var
  
    bn_param[2].data[...] = np.array([1.0])  
    conv_param[0].data[...] = np.reshape(buf[start:start+conv_weight.size], conv_weight.shape); start = start + conv_weight.size  
    #print conv_weight.size
    #print conv_weight
  
    return start
def darknet2caffe(cfgfile, weightfile, protofile, caffemodel='gene.caffemodel'):  
    #net_info = cfg2prototxt(cfgfile)
    #save_prototxt(net_info , protofile, region=False)  
    print('benchmark')
    net = caffe.Net(protofile, caffe.TEST)
    k_v_s = [(k, v) for k, v in net.params.items()]
    key_vecnums = [(vecs[0], len(vecs[1])) for vecs in k_v_s]
    print([(vecs[0], [vecs[1][i].data.shape for i in range(len(vecs[1]))])for vecs in k_v_s])
    print('benchmark')
    params = net.params
    print('benchmark')
  
    #blocks = parse_cfg(cfgfile)
    parser = ConfigParser(dict_type=uniqdict)
    parser.read(cfgfile)
    blocks = parser.sections()
    print(blocks)
    
  
    #Open the weights file  
    fp = open(weightfile, "rb")  
  
    #The first 4 values are header information   
    # 1. Major version number  
    # 2. Minor Version Number  
    # 3. Subversion number   
    # 4. IMages seen   
    header = np.fromfile(fp, dtype = np.int32, count = 5)
    #header = np.fromfile(fp, dtype = np.float32, count = 5)
    print(header)
    #fp = open(weightfile, 'rb')  
    #header = np.fromfile(fp, count=5, dtype=np.int32)  
    #header = np.ndarray(shape=(5,),dtype='int32',buffer=fp.read(20))  
    #print(header)  
    buf = np.fromfile(fp, dtype = np.float32)
    print('buf len:{0}'.format(len(buf)))
    #print(buf)  
    fp.close()  
  
    layers = []  
    layer_id = 1  
    start = 0  
    for block in blocks:
        print(block)
        if start >= buf.size:  
            break
        items = dict(parser.items(block))
        print(items)
        if block.split('_')[0] == 'net':  
            continue
        elif ((block.split('_')[0] == 'convolutional') or 
        (block.split('_')[0] == 'deconvolutional')):
            batchnorm_followed = False
            relu_followed = False
            
            if 'batch_normalize' in items and items['batch_normalize']:
                batchnorm_followed = True
            if 'activation' in items and items['activation'] != 'linear':
                relu_followed = True
            
            if items.has_key('name'):  
                conv_layer_name = items['name']  
                print('has key name ' + conv_layer_name)
                bn_layer_name = '%s-bn' % items['name']  
                scale_layer_name = '%s-scale' % items['name']  
            else:
                if(block.split('_')[0] == 'deconvolutional'):
                    conv_layer_name = 'layer%d-upsample' % layer_id  
                    print('has no name ' + conv_layer_name)
                    #bn_layer_name = 'layer%d-bn' % layer_id  
                    #scale_layer_name = 'layer%d-scale' % layer_id 
                else:
                    conv_layer_name = 'layer%d-conv' % layer_id  
                    print('has no name ' + conv_layer_name)
                    bn_layer_name = 'layer%d-bn' % layer_id  
                    scale_layer_name = 'layer%d-scale' % layer_id  
  
            if batchnorm_followed:
                print("load_conv_bn2caffe:")
                start = load_conv_bn2caffe(buf, start, params[conv_layer_name], 
                                           params[bn_layer_name], params[scale_layer_name])
            else:
                print("load_conv2caffe:")
                start = load_conv2caffe(buf, start, params[conv_layer_name])
            '''
            if(layer_id == 11):
                print('layer_id == 11')
                layer_id = layer_id + 2
            else:
                layer_id = layer_id+1
            '''
            layer_id = layer_id+1
            print('start:{0}'.format(start))
        elif block.split('_')[0] == 'connected':  
            if items.has_key('name'):  
                fc_layer_name = items['name']  
            else:  
                fc_layer_name = 'layer%d-fc' % layer_id  
            start = load_fc2caffe(buf, start, params[fc_layer_name])  
            layer_id = layer_id+1  
        elif block.split('_')[0] == 'maxpool':  
            layer_id = layer_id+1  
        elif block.split('_')[0] == 'avgpool':  
            layer_id = layer_id+1  
        elif block.split('_')[0] == 'region':  
            layer_id = layer_id + 1  
        elif block.split('_')[0] == 'route':  
            layer_id = layer_id + 1  
        elif block.split('_')[0] == 'shortcut':  
            layer_id = layer_id + 1  
        elif block.split('_')[0] == 'softmax':  
            layer_id = layer_id + 1  
        elif block.split('_')[0] == 'cost':  
            layer_id = layer_id + 1  
        elif block.split('_')[0] == 'upsample':  
            layer_id = layer_id + 1 
        else:  
            print('unknow layer type %s ' % block.split('_')[0]) 
            layer_id = layer_id + 1 
    print('save caffemodel to %s' % caffemodel)  
    net.save(caffemodel)
    
#cfgfile = '/data/github_repos/yolov3-tiny-fit-ncs/ncs/yolov3-tiny-ncs-without-last-maxpool.cfg'
#weights = weights
prototxt = '/data/github_repos/yolov3-tiny-fit-ncs/ncs/yolov3-tiny-ncs-without-last-maxpool.prototxt'
prototxt = saved_prototxt
prototxt1 = '/data/github_repos/yolov3-tiny-fit-ncs/ncs/convert_tools/07:30:49.817376yolov3-tiny-ncs-without-last-maxpool.cfg.prototxt'
#caffemodel = 'Jenerated_nolastpooling.caffemodel'
caffemodel = (datetime.datetime.now()).strftime("%Y%m%d%H%M%S_") + 'TinyYoloV3NCS.caffemodel'
darknet2caffe(cfgfile, weights, prototxt, caffemodel)
make_prototxt_command = 'cp ' + prototxt +' ' + caffemodel[:-10] + 'prototxt'
prototxt = caffemodel[:-10] + 'prototxt'
os.system(make_prototxt_command)
#mvnc_command = 'bash generate-graph.sh ' + caffemodel + ' ' + prototxt1
#os.system(mvnc_command)
set_ncs2_env_command = 'bash /opt/intel/openvino/bin/setupvars.sh'
#set_ncs2_env_command = 'bash /opt/intel/computer_vision_sdk/bin/setupvars.sh'
os.system(set_ncs2_env_command)
mo_command = 'sh /data/github_repos/yolov3-tiny-fit-ncs/ncs2/OpenVINO/model_optimizer/yolov3-tiny-mo.sh ' + '/data/github_repos/yolov3-tiny-fit-ncs/ncs/convert_tools/' + caffemodel
#mo_command = 'sh /data/github_repos/yolov3-tiny-fit-ncs/ncs2/OpenVINO/model_optimizer/yolov3-tiny-mo-2018R5.sh ' + '/data/github_repos/yolov3-tiny-fit-ncs/ncs/convert_tools/' + caffemodel

os.system(mo_command)

benchmark
[('layer1-conv', [(16, 3, 3, 3)]), ('layer1-bn', [(16,), (16,), (1,)]), ('layer1-scale', [(16,), (16,)]), ('layer3-conv', [(32, 16, 3, 3)]), ('layer3-bn', [(32,), (32,), (1,)]), ('layer3-scale', [(32,), (32,)]), ('layer5-conv', [(64, 32, 3, 3)]), ('layer5-bn', [(64,), (64,), (1,)]), ('layer5-scale', [(64,), (64,)]), ('layer7-conv', [(128, 64, 3, 3)]), ('layer7-bn', [(128,), (128,), (1,)]), ('layer7-scale', [(128,), (128,)]), ('layer9-conv', [(256, 128, 3, 3)]), ('layer9-bn', [(256,), (256,), (1,)]), ('layer9-scale', [(256,), (256,)]), ('layer11-conv', [(512, 256, 3, 3)]), ('layer11-bn', [(512,), (512,), (1,)]), ('layer11-scale', [(512,), (512,)]), ('layer12-conv', [(1024, 512, 3, 3)]), ('layer12-bn', [(1024,), (1024,), (1,)]), ('layer12-scale', [(1024,), (1024,)]), ('layer13-conv', [(256, 1024, 1, 1)]), ('layer13-bn', [(256,), (256,), (1,)]), ('layer13-scale', [(256,), (256,)]), ('layer14-conv', [(512, 256, 3, 3)]), ('layer14-bn', [(512,), (512,), (1,)]), ('layer14-scale', [(

0